# Basketball IQ Analysis: LeBron James

Comprehensive analysis of Basketball IQ indicators using NBA API data for LeBron James (ID: 2544).

In [245]:
import pandas as pd
import numpy as np

# LeBron's player ID
lebron_id = "2544"

# Load basic stats
top200 = pd.read_csv('../data/raw/top200_per.csv')
lebron_basic = top200[top200['PLAYER_ID'] == int(lebron_id)].iloc[0]

print("=== LEBRON BASIC STATS ===")
print(f"Player: {lebron_basic['PLAYER']}")
print(f"Team: {lebron_basic['TEAM']}")
print(f"PPG: {lebron_basic['PTS']}")
print(f"AST: {lebron_basic['AST']}")
print(f"TOV: {lebron_basic['TOV']}")
print(f"Games Played: {lebron_basic['GP']}")

=== LEBRON BASIC STATS ===
Player: LeBron James
Team: LAL
PPG: 24.4
AST: 8.2
TOV: 3.7
Games Played: 70


In [246]:
# 1. Assist-to-Turnover Ratio
ast_tov_ratio = lebron_basic['AST'] / lebron_basic['TOV']

print(f"\n1. Assist-to-Turnover Ratio: {ast_tov_ratio:.3f}")
print(f"   Assists: {lebron_basic['AST']}, Turnovers: {lebron_basic['TOV']}")
print(f"   Interpretation: High ratio indicates smart decisions under pressure")
print(f"   League elite threshold: ~3.5+, LeBron: {ast_tov_ratio:.1f}")


1. Assist-to-Turnover Ratio: 2.216
   Assists: 8.2, Turnovers: 3.7
   Interpretation: High ratio indicates smart decisions under pressure
   League elite threshold: ~3.5+, LeBron: 2.2


In [247]:
# 2. Late Clock Efficiency
shot_clock_data = pd.read_csv(f'../data/raw/{lebron_id}_DribbleShooting.csv')

print("\n2. Late Clock Efficiency (Composure Under Pressure)")
print("Shot Clock Data:")
print(shot_clock_data[['SHOT_CLOCK_RANGE', 'FGA_FREQUENCY', 'FG_PCT']].round(3))

# Calculate late clock efficiency (7-4 Late + 4-0 Very Late)
late_clock = shot_clock_data[shot_clock_data['SHOT_CLOCK_RANGE'] == '7-4 Late']['FG_PCT'].iloc[0]
very_late_clock = shot_clock_data[shot_clock_data['SHOT_CLOCK_RANGE'] == '4-0 Very Late']['FG_PCT'].iloc[0]

# Weight by frequency for combined late clock efficiency
late_freq = shot_clock_data[shot_clock_data['SHOT_CLOCK_RANGE'] == '7-4 Late']['FGA_FREQUENCY'].iloc[0]
very_late_freq = shot_clock_data[shot_clock_data['SHOT_CLOCK_RANGE'] == '4-0 Very Late']['FGA_FREQUENCY'].iloc[0]

combined_late_efficiency = (late_clock * late_freq + very_late_clock * very_late_freq) / (late_freq + very_late_freq)

print(f"\nLate Clock (7-4 sec) FG%: {late_clock:.3f}")
print(f"Very Late Clock (4-0 sec) FG%: {very_late_clock:.3f}")
print(f"Combined Late Clock Efficiency: {combined_late_efficiency:.3f}")
print(f"Interpretation: Higher efficiency in pressure situations shows composure and smart shot selection")


2. Late Clock Efficiency (Composure Under Pressure)
Shot Clock Data:
   SHOT_CLOCK_RANGE  FGA_FREQUENCY  FG_PCT
0             24-22          0.020   0.760
1  22-18 Very Early          0.176   0.665
2       18-15 Early          0.119   0.503
3      15-7 Average          0.457   0.491
4          7-4 Late          0.118   0.407
5     4-0 Very Late          0.109   0.439

Late Clock (7-4 sec) FG%: 0.407
Very Late Clock (4-0 sec) FG%: 0.439
Combined Late Clock Efficiency: 0.422
Interpretation: Higher efficiency in pressure situations shows composure and smart shot selection


In [248]:
# 3. Clutch AST/TOV (Decision quality in pressure situations)
clutch_stats = pd.read_csv('../data/raw/league_clutch_stats.csv')

# Find LeBron in clutch stats
lebron_clutch = clutch_stats[clutch_stats['PLAYER_ID'] == int(lebron_id)]

if not lebron_clutch.empty:
    lebron_clutch = lebron_clutch.iloc[0]
    clutch_ast_tov = lebron_clutch['AST'] / lebron_clutch['TOV'] if lebron_clutch['TOV'] > 0 else float('inf')
    
    print(f"\n3. Clutch Decision Quality (AST/TOV in close games)")
    print(f"   Clutch Assists: {lebron_clutch['AST']:.1f}")
    print(f"   Clutch Turnovers: {lebron_clutch['TOV']:.1f}")
    print(f"   Clutch AST/TOV Ratio: {clutch_ast_tov:.3f}")
    print(f"   Interpretation: Decision-making quality when games are on the line")
else:
    print(f"\n3. Clutch Decision Quality: No clutch data available for LeBron")


3. Clutch Decision Quality (AST/TOV in close games)
   Clutch Assists: 6.8
   Clutch Turnovers: 2.5
   Clutch AST/TOV Ratio: 2.720
   Interpretation: Decision-making quality when games are on the line


In [249]:
# 4. Effective Field Goal % (Shot Selection Intelligence)
fg_pct = lebron_basic['FG_PCT']
fg3_pct = lebron_basic['FG3_PCT'] 
fg3a_per_fga = lebron_basic['FG3A'] / lebron_basic['FGA']

# Calculate eFG% = (FGM + 0.5 * 3PM) / FGA
efg_pct = (lebron_basic['FGM'] + 0.5 * lebron_basic['FG3M']) / lebron_basic['FGA']

print(f"\n4. Shot Selection Intelligence (Effective Field Goal %)")
print(f"   FG%: {fg_pct:.3f}")
print(f"   3P%: {fg3_pct:.3f}")
print(f"   3PA/FGA: {fg3a_per_fga:.3f}")
print(f"   Effective FG%: {efg_pct:.3f}")
print(f"   Interpretation: Higher eFG% indicates smart shot selection and recognition of good opportunities")


4. Shot Selection Intelligence (Effective Field Goal %)
   FG%: 0.513
   3P%: 0.376
   3PA/FGA: 0.315
   Effective FG%: 0.572
   Interpretation: Higher eFG% indicates smart shot selection and recognition of good opportunities


In [250]:
# 5. Deflections per 36 (Anticipation and Reading the Game)
hustle_stats = pd.read_csv('../data/raw/league_hustle_stats.csv')
lebron_hustle = hustle_stats[hustle_stats['PLAYER_ID'] == int(lebron_id)]

if not lebron_hustle.empty:
    lebron_hustle = lebron_hustle.iloc[0]
    # Data is already per 36, no need to normalize
    deflections_per_36 = lebron_hustle['DEFLECTIONS']

    print(f"\n5. Deflections per 36 (Anticipation & Game Reading)")
    print(f"   Deflections per 36: {deflections_per_36:.2f}")
    print(f"   Interpretation: Higher rate shows anticipation and ability to read offensive patterns")
else:
    print(f"\n5. Deflections per 36: No hustle data available for LeBron")


5. Deflections per 36 (Anticipation & Game Reading)
   Deflections per 36: 1.90
   Interpretation: Higher rate shows anticipation and ability to read offensive patterns


In [251]:
# 6. Screen Assists per 36 (Off-ball Intelligence)
if not lebron_hustle.empty:
    # Data is already per 36, no need to normalize
    screen_assists_per_36 = lebron_hustle['SCREEN_ASSISTS']

    print(f"\n6. Screen Assists per 36 (Off-ball Intelligence)")
    print(f"   Screen Assists per 36: {screen_assists_per_36:.2f}")
    print(f"   Interpretation: Shows understanding of how to create opportunities for teammates through positioning")
else:
    print(f"\n6. Screen Assists per 36: No hustle data available for LeBron")


6. Screen Assists per 36 (Off-ball Intelligence)
   Screen Assists per 36: 0.90
   Interpretation: Shows understanding of how to create opportunities for teammates through positioning


In [252]:
# 7. Quick Decision Efficiency (Success rate when forced to decide quickly)
touch_time_data = pd.read_csv(f'../data/raw/{lebron_id}_TouchTimeShooting.csv')

print(f"\n7. Quick Decision Efficiency (Success when forced to decide quickly)")
print("Touch Time Data:")
print(touch_time_data[['TOUCH_TIME_RANGE', 'FGA_FREQUENCY', 'FG_PCT']].round(3))

quick_decision_efficiency = touch_time_data[touch_time_data['TOUCH_TIME_RANGE'] == 'Touch < 2 Seconds']['FG_PCT'].iloc[0]
quick_decision_frequency = touch_time_data[touch_time_data['TOUCH_TIME_RANGE'] == 'Touch < 2 Seconds']['FGA_FREQUENCY'].iloc[0]

# Calculate frequency-weighted quick decision impact
quick_decision_impact = quick_decision_frequency * quick_decision_efficiency

# Calculate overall weighted efficiency for comparison
total_weighted_efficiency = (touch_time_data['FGA_FREQUENCY'] * touch_time_data['FG_PCT']).sum()
quick_decision_relative_impact = quick_decision_impact/total_weighted_efficiency

print(f"\nQuick Decision (<2 sec) Efficiency: {quick_decision_efficiency:.3f}")
print(f"Quick Decision Frequency: {quick_decision_frequency:.3f}")
print(f"Quick Decision Impact (Freq × Eff): {quick_decision_impact:.3f}")
print(f"Overall Weighted Efficiency: {total_weighted_efficiency:.3f}")
print(f"Quick Decision Relative Impact: {quick_decision_impact/total_weighted_efficiency:.3f}")
print(f"Interpretation: {100 * quick_decision_relative_impact:.2f}% of Lebron's overall shooting value comes from quick decisions.")


7. Quick Decision Efficiency (Success when forced to decide quickly)
Touch Time Data:
    TOUCH_TIME_RANGE  FGA_FREQUENCY  FG_PCT
0  Touch < 2 Seconds          0.334   0.575
1  Touch 2-6 Seconds          0.328   0.463
2   Touch 6+ Seconds          0.338   0.499

Quick Decision (<2 sec) Efficiency: 0.575
Quick Decision Frequency: 0.334
Quick Decision Impact (Freq × Eff): 0.192
Overall Weighted Efficiency: 0.513
Quick Decision Relative Impact: 0.375
Interpretation: 37.47% of Lebron's overall shooting value comes from quick decisions.


In [253]:
# 8. Loose Balls Recovered per 36 (Defensive Timing and Hustle)
if not lebron_hustle.empty:
    loose_balls_per_36 = lebron_hustle['LOOSE_BALLS_RECOVERED']
    
    print(f"\n8. Loose Balls Recovered per 36 (Defensive Timing & Hustle)")
    print(f"   Loose Balls per 36: {loose_balls_per_36:.2f}")
    print(f"   Interpretation: Shows anticipation, positioning, and effort on 50/50 balls")
else:
    print("=== RISK MANAGEMENT IQ ===")
    print(f"\n8. Loose Balls Recovered per 36: No hustle data available for LeBron")


8. Loose Balls Recovered per 36 (Defensive Timing & Hustle)
   Loose Balls per 36: 0.70
   Interpretation: Shows anticipation, positioning, and effort on 50/50 balls


In [254]:
# 9. Shooting Foul Percentage (Defensive Discipline)
general_splits = pd.read_csv(f'../data/raw/{lebron_id}_general_splits.csv')
closest_defender = pd.read_csv(f'../data/raw/{lebron_id}_ClosestDefenderShooting.csv')

if not general_splits.empty and not closest_defender.empty:
    # Use overall stats row (total season stats)
    overall_stats = general_splits.iloc[0]

    personal_fouls = overall_stats['PF']
    minutes = overall_stats['MIN']
    games_played = overall_stats['GP']

    # Calculate total shots defended (sum FGA from all shot types, multiply by games)
    shots_defended_per_game = closest_defender['FGA'].sum()
    total_shots_defended = shots_defended_per_game * games_played

    print(f"lebron defended {total_shots_defended} shots in the season")

    # Assume shooting fouls are one third of all personal fouls
    shooting_fouls = personal_fouls * 0.333

    # Calculate shooting foul percentage
    shooting_foul_percentage = (shooting_fouls / total_shots_defended * 100) if total_shots_defended > 0 else 0

    print(f"\n9. Shooting Foul Percentage")
    print(f"   Total shots defended: {total_shots_defended:.0f}")
    print(f"   Estimated shooting fouls: {shooting_fouls:.1f}")
    print(f"   Shooting Foul Percentage: {shooting_foul_percentage:.1f}%")
    print(f"   Interpretation: Lower percentage shows better defensive discipline")
else:
    print(f"\n9. Shooting Foul Percentage: Data not available for LeBron")

lebron defended 616.76 shots in the season

9. Shooting Foul Percentage
   Total shots defended: 617
   Estimated shooting fouls: 14.7
   Shooting Foul Percentage: 2.4%
   Interpretation: Lower percentage shows better defensive discipline


In [255]:
# 10. Successful Box Outs per 36 (Rebounding Positioning and Effort)
if not lebron_hustle.empty:
    successful_boxouts_per_36 = lebron_hustle['BOX_OUT_PLAYER_REBS']
    total_boxouts_per_36 = lebron_hustle['BOX_OUTS']
    boxout_success_rate = lebron_hustle['BOX_OUT_PLAYER_REBS'] / lebron_hustle['BOX_OUTS'] if lebron_hustle['BOX_OUTS'] > 0 else 0
    
    print(f"\n10. Successful Box Outs per 36 (Rebounding Positioning & Effort)")
    print(f"   Rebounds after Box Outs per 36: {successful_boxouts_per_36:.2f}")
    print(f"   Total Box Outs per 36: {total_boxouts_per_36:.2f}")
    print(f"   Box Out Success Rate: {boxout_success_rate:.3f}")
    print(f"   Interpretation: Shows both positioning effort and effectiveness in securing rebounds")
else:
    print(f"\n10. Successful Box Outs per 36: No hustle data available for LeBron")


10. Successful Box Outs per 36 (Rebounding Positioning & Effort)
   Rebounds after Box Outs per 36: 0.40
   Total Box Outs per 36: 0.60
   Box Out Success Rate: 0.667
   Interpretation: Shows both positioning effort and effectiveness in securing rebounds


In [256]:
# 11. Charges Drawn per 36 (Risk vs Reward Decision Making)
if not lebron_hustle.empty:
    charges_per_36 = lebron_hustle['CHARGES_DRAWN']

    print(f"\n11. Charges Drawn per 36 (Risk vs Reward Decision Making)")
    print(f"   Charges Drawn per 36: {charges_per_36:.2f}")
    print(f"   Interpretation: Shows willingness to take calculated risks for defensive advantage")
else:
    print(f"\n11. Charges Drawn per 36: No hustle data available for LeBron")


11. Charges Drawn per 36 (Risk vs Reward Decision Making)
   Charges Drawn per 36: 0.10
   Interpretation: Shows willingness to take calculated risks for defensive advantage


In [257]:
# 12. Corrected Personalized Shot Selection Intelligence (Using Shooting % Data)
closest_def_shooting = pd.read_csv(f'../data/raw/{lebron_id}_ClosestDefenderShooting.csv')

if not closest_def_shooting.empty:
    print(f"\n12. Personalized Shot Selection Intelligence (Corrected)")
    print("Available shooting data with actual percentages:")
    print(closest_def_shooting[['SHOT_TYPE', 'FGA_FREQUENCY', 'FG_PCT', 'EFG_PCT']])

    # Extract key zones from available data
    close_shots = closest_def_shooting[closest_def_shooting['SHOT_TYPE'] == 'Less than 10 ft'].iloc[0]
    catch_shoot = closest_def_shooting[closest_def_shooting['SHOT_TYPE'] == 'Catch and Shoot'].iloc[0]
    pull_ups = closest_def_shooting[closest_def_shooting['SHOT_TYPE'] == 'Pull Ups'].iloc[0]

    print(f"\nShot Selection Analysis:")
    print(f"\nAt Rim/Close Range (< 10 ft):")
    print(f"   Frequency: {close_shots['FGA_FREQUENCY']:.1%}")
    print(f"   FG%: {close_shots['FG_PCT']:.3f}")
    print(f"   Points per Shot: {close_shots['FG_PCT'] * 2:.3f}")

    print(f"\nCatch and Shoot:")
    print(f"   Frequency: {catch_shoot['FGA_FREQUENCY']:.1%}")
    print(f"   FG%: {catch_shoot['FG_PCT']:.3f}")
    print(f"   eFG%: {catch_shoot['EFG_PCT']:.3f}")

    print(f"\nPull Up Shots:")
    print(f"   Frequency: {pull_ups['FGA_FREQUENCY']:.1%}")
    print(f"   FG%: {pull_ups['FG_PCT']:.3f}")
    print(f"   eFG%: {pull_ups['EFG_PCT']:.3f}")

    # Basic shot selection score (higher frequency on higher efficiency shots)
    close_range_value = close_shots['FGA_FREQUENCY'] * (close_shots['FG_PCT'] * 2)
    catch_shoot_value = catch_shoot['FGA_FREQUENCY'] * catch_shoot['EFG_PCT']
    pull_up_value = pull_ups['FGA_FREQUENCY'] * pull_ups['EFG_PCT']

    total_shot_value = close_range_value + catch_shoot_value + pull_up_value

    print(f"\nShot Selection Efficiency:")
    print(f"   Close Range Value: {close_range_value:.3f}")
    print(f"   Catch & Shoot Value: {catch_shoot_value:.3f}")
    print(f"   Pull Up Value: {pull_up_value:.3f}")
    print(f"   Total Shot Selection Value: {total_shot_value:.3f}")
    print(f"   Interpretation: Higher values indicate better shot selection for this player's skillset")
else:
    print(f"\n12. Personalized Shot Selection: No shooting data available for LeBron")


12. Personalized Shot Selection Intelligence (Corrected)
Available shooting data with actual percentages:
         SHOT_TYPE  FGA_FREQUENCY  FG_PCT  EFG_PCT
0  Catch and Shoot          0.178   0.429    0.619
1         Pull Ups          0.337   0.386    0.451
2  Less than 10 ft          0.461   0.641    0.641
3            Other          0.024   0.452    0.565

Shot Selection Analysis:

At Rim/Close Range (< 10 ft):
   Frequency: 46.1%
   FG%: 0.641
   Points per Shot: 1.282

Catch and Shoot:
   Frequency: 17.8%
   FG%: 0.429
   eFG%: 0.619

Pull Up Shots:
   Frequency: 33.7%
   FG%: 0.386
   eFG%: 0.451

Shot Selection Efficiency:
   Close Range Value: 0.591
   Catch & Shoot Value: 0.110
   Pull Up Value: 0.152
   Total Shot Selection Value: 0.853
   Interpretation: Higher values indicate better shot selection for this player's skillset


In [258]:
# Load the complete IQ metrics dataset to calculate LeBron's percentiles
iq_metrics = pd.read_csv('../data/processed/all_player_iq_metrics.csv')

# Find LeBron's data
lebron_metrics = iq_metrics[iq_metrics['PLAYER_ID'] == int(lebron_id)].iloc[0]

# Calculate percentiles for each IQ metric
metric_columns = [
    'ast_tov_ratio', 'late_clock_efficiency', 'clutch_ast_tov', 'efg_pct',
    'deflections_per_36', 'screen_assists_per_36', 'quick_decision_efficiency',
    'loose_balls_per_36', 'shooting_foul_pct', 'successful_boxouts_per_36',
    'charges_drawn_per_36', 'shot_selection_value'
]

# Create LeBron's percentile dataframe
lebron_percentiles = []

for metric in metric_columns:
    # Get LeBron's value
    lebron_value = lebron_metrics[metric]
    
    # Calculate percentile (for shooting_foul_pct, lower is better, so we flip it)
    if metric == 'shooting_foul_pct':
        # For shooting fouls, lower is better, so we calculate what % he's better than
        percentile = (iq_metrics[metric] > lebron_value).mean() * 100
    else:
        # For all other metrics, higher is better
        percentile = (iq_metrics[metric] < lebron_value).mean() * 100
    
    # Get league average and rank
    league_avg = iq_metrics[metric].mean()
    rank = iq_metrics[metric].rank(ascending=False if metric != 'shooting_foul_pct' else True)[iq_metrics['PLAYER_ID'] == int(lebron_id)].iloc[0]
    total_players = len(iq_metrics)
    
    lebron_percentiles.append({
        'Metric': metric.replace('_', ' ').title(),
        'LeBron_Value': round(lebron_value, 3),
        'League_Average': round(league_avg, 3),
        'Percentile': round(percentile, 1),
        'Rank': f"{int(rank)}/{total_players}"
    })

# Convert to DataFrame
percentiles_df = pd.DataFrame(lebron_percentiles)

print("LeBron's Basketball IQ Percentiles vs League:")
display(percentiles_df)

LeBron's Basketball IQ Percentiles vs League:


,Metric,LeBron_Value,League_Average,Percentile,Rank
0,Ast Tov Ratio,2.216,2.062,67.8,65/202
1,Late Clock Efficiency,0.422,0.403,68.3,63/202
2,Clutch Ast Tov,2.720,1.915,79.2,42/202
3,Efg Pct,0.572,0.550,68.8,62/202
4,Deflections Per 36,1.900,2.401,27.7,140/202
5,Screen Assists Per 36,0.900,1.091,71.3,56/202
6,Quick Decision Efficiency,0.575,0.471,89.1,21/202
7,Loose Balls Per 36,0.700,0.705,40.6,99/202
8,Shooting Foul Pct,6.810,23.823,95.0,10/202
9,Successful Boxouts Per 36,0.400,0.461,64.9,63/202
